# Setup

## Verify we're in the Conda environment

In [ ]:
import sys
print(sys.executable)

## Import python packages

In [ ]:
import os
import sys
import json
import openai
from PIL import Image
import base64
import io
from dotenv import load_dotenv
import requests
from openai import OpenAI
import pprint
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import subprocess
import textwrap
from collections import Counter
import pprint
import random
from collections import defaultdict

## Function to create_thumbnails for each storyBook

In [ ]:
import os
import json
import shutil
from PIL import Image

# Configuration
input_json = "storybookNFTs.json"
image_base_dir = Path("./images_charles")
output_dir = Path("./images_charles_nft_thumbnails")
thumbnail_size = (150, 150)

# Delete the folder and all its contents if it exists
shutil.rmtree(output_dir)

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Load JSON data
with open(input_json, "r") as file:
    data = json.load(file)

def process_images(data):
    index = 0  # Counter for naming output files
    for page in data:
        # Extract the first doublepage item
        first_item = page.get("doublepages", [None])[0]
        if not first_item:
            continue  # Skip if no valid doublepage

        # Extract image path
        image_url = first_item.get("imageUrl")
        if not image_url:
            print(f"No imageUrl found for index {index}")
            continue

        image_path = os.path.join(image_base_dir, os.path.basename(image_url))

        # Process the image
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            continue

        try:
            # Open the image and convert to thumbnail
            with Image.open(image_path) as img:
                img = img.convert("RGB")  # Ensure image is in RGB mode

                # Make the image square by cropping
                min_side = min(img.size)
                img = img.crop((
                    (img.width - min_side) // 2,
                    (img.height - min_side) // 2,
                    (img.width + min_side) // 2,
                    (img.height + min_side) // 2,
                ))

                # Resize to thumbnail size
                img.thumbnail(thumbnail_size)

                # Save image with zero-padded index
                output_file = os.path.join(output_dir, f"{index:03}.png")
                img.save(output_file, format="PNG", optimize=True)
                print(f"Thumbnail saved: {output_file}")

        except Exception as e:
            print(f"Error processing image {image_path}: {e}")

        index += 1

# Run the processing
process_images(data)